# EUR-Lex Data Exploration

This notebook creates a database from the scraped JSON files and performs exploratory data analysis.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import sqlite3
from tqdm.notebook import tqdm

# Set style for plots
plt.style.use('seaborn')
sns.set_palette('husl')

## Create SQLite Database

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('eurlex_documents.db')
cursor = conn.cursor()

# Create table for documents
cursor.execute("""
CREATE TABLE IF NOT EXISTS documents (
    celex TEXT PRIMARY KEY,
    title TEXT,
    document_type TEXT,
    year INTEGER,
    number INTEGER,
    date_document DATE,
    date_effect DATE,
    date_end DATE,
    directory_code TEXT,
    full_text TEXT
)
""")

conn.commit()

In [ ]:
# Function to load JSON files into database
def load_json_files():
    data_dir = Path('../data')
    json_files = list(data_dir.rglob('*.json'))
    
    for file in tqdm(json_files, desc='Loading documents'):
        with open(file, 'r') as f:
            try:
                data = json.load(f)
                
                # Extract data
                cursor.execute("""
                INSERT OR REPLACE INTO documents 
                (celex, title, document_type, year, number, date_document, 
                 date_effect, date_end, directory_code, full_text)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    data.get('celex'),
                    data.get('title'),
                    data.get('document_type'),
                    data.get('year'),
                    data.get('number'),
                    data.get('date_document'),
                    data.get('date_effect'),
                    data.get('date_end'),
                    data.get('directory_code'),
                    data.get('full_text')
                ))
                
            except json.JSONDecodeError:
                print(f"Error loading {file}")
                continue
    
    conn.commit()

# Load the data
load_json_files()

## Basic Data Analysis

In [ ]:
# Load data into pandas
df = pd.read_sql("""
SELECT 
    document_type,
    year,
    date_document,
    date_effect,
    directory_code
FROM documents
""", conn)

# Convert dates
for col in ['date_document', 'date_effect']:
    df[col] = pd.to_datetime(df[col])

print("Total number of documents:", len(df))
print("\nDocument types distribution:")
print(df['document_type'].value_counts())

## Time Series Analysis

In [ ]:
# Documents per year
yearly_counts = df['year'].value_counts().sort_index()

plt.figure(figsize=(15, 6))
yearly_counts.plot(kind='bar')
plt.title('Number of Documents per Year')
plt.xlabel('Year')
plt.ylabel('Number of Documents')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Document Type Analysis

In [ ]:
# Document types over time
doc_types_by_year = df.pivot_table(
    index='year',
    columns='document_type',
    aggfunc='size',
    fill_value=0
)

plt.figure(figsize=(15, 8))
doc_types_by_year.plot(kind='area', stacked=True)
plt.title('Document Types Distribution Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Documents')
plt.legend(title='Document Type', bbox_to_anchor=(1.05, 1))
plt.tight_layout()
plt.show()

## Directory Code Analysis

In [ ]:
# Top directory codes
print("Most common directory codes:")
print(df['directory_code'].value_counts().head(10))

# Plot top directory codes
plt.figure(figsize=(12, 6))
df['directory_code'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 Directory Codes')
plt.xlabel('Directory Code')
plt.ylabel('Number of Documents')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()